In [1]:
%%capture
!pip install polars
!pip install lightgbm==4.2.0
!pip install catboost==1.2.7
!pip install xgboost==2.0.3
!pip install optuna
!pip install lifelines

In [2]:
import shutil
import pandas as pd
import numpy as np
import re
import gc
import os
import math
import importlib
import random
import warnings
import pickle
import joblib
from tqdm import tqdm
import sys

from scipy.stats import skew, kurtosis
from collections import Counter,defaultdict
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import lightgbm as lgb
import catboost as cat, gc
import xgboost as xgb

print(lgb.__version__)
print(cat.__version__)
print(xgb.__version__)

from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier, Pool
from sklearn.svm import SVR
from xgboost import XGBRegressor, XGBClassifier

import optuna

from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss


import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts
import torch.nn.functional as F

from transformers import AutoModel,AutoTokenizer

warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')


4.2.0
1.2.7
2.0.3


In [29]:
sys.path.append('/notebooks/CIBMTR/utils')
sys.path.append('/notebooks/CIBMTR/train')

import tuning
import utils as my_utils
import metric
from metric import score
from utils import clr
import train as trainer
import config

# **LOAD DATA**

In [6]:
train_path = '/notebooks/CIBMTR/data/train.csv'
train_dic_path = '/notebooks/CIBMTR/data/data_dictionary.csv'

test_path = '/notebooks/CIBMTR/data/test.csv'
sub_path = '/notebooks/CIBMTR/data/sample_submission.csv'

In [7]:
train = pd.read_csv(train_path)
# dictionary_tow = pd.read_csv(train_dic_path)
dictionary = pd.read_csv('/notebooks/CIBMTR/output/dictionary.csv', encoding='cp932').drop('Unnamed: 0', axis=1)
test = pd.read_csv(test_path)
submission = pd.read_csv(sub_path)

# **CALCULATE CV**

In [24]:
data = train

In [27]:
EXP_NUM = 'EXP-3'
SAVE_PATH = f'/notebooks/CIBMTR/output/{EXP_NUM}'

oof_preds = joblib.load(f'{SAVE_PATH}/oof_preds_{EXP_NUM}.pkl')
oof_preds.shape

(28800,)

In [28]:
y_true_fold = data[['ID', 'efs', 'efs_time', 'race_group']].copy()
y_pred_fold = data[['ID']].copy()
y_pred_fold['prediction'] = oof_preds
loss_val = score(y_true_fold, y_pred_fold, 'ID')
print(clr.RED + f'Overall C-index : {loss_val:.4}' + clr.END)

C-index-American Indian or Alaska Native : 0.686
C-index-Asian : 0.695
C-index-Black or African-American : 0.670
C-index-More than one race : 0.686
C-index-Native Hawaiian or other Pacific Islander : 0.674
C-index-White : 0.664
Overall C-index : 0.6686
